#Investing in Competitive Advantage: The Impact of Advertising and R&D Intensity on Firm Performance

**Author:** Oguz Alp Eren

The code below was developed for creating the models in my master's thesis. It's important to note that the model numbers are not consistent between the thesis document and this code document. The code was written progressing from the simplest to the most complex model and this progression was not a consideration in the thesis. Explanations for the code are provided within the cells and are sometimes accompanied by additional text boxes for extra clarity.

Each model, except for the separate industry fixed effects models, is run twice: once with robust standard errors and once without. All models showed significant results in the Breusch-Pagan test. This necessitated the use of robust standard errors.

#Preprocessing

In [ ]:
!pip install linearmodels

In [ ]:
import io
import pandas as pd
from linearmodels.panel import PanelOLS, RandomEffects
from linearmodels.panel import compare

In [ ]:
df = pd.read_csv('Thesis File - Fortune 500.csv')
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,xsga,xstf,xstfo,xstfws,costat,prcc_c,mkvalt,prcc_f,gind,spcindcd
0,1045,2012-12-31,2012,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,2892.0,NaN,NaN,NaN,A,0.795,266.5571,0.795,203020,605.0
1,1045,2013-12-31,2013,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,4672.0,NaN,NaN,NaN,A,25.250,6591.9923,25.250,203020,605.0
2,1045,2014-12-31,2014,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,6554.0,NaN,NaN,NaN,A,53.630,37405.5843,53.630,203020,605.0
3,1045,2015-12-31,2015,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,6683.0,NaN,NaN,NaN,A,42.350,26452.7417,42.350,203020,605.0
4,1045,2016-12-31,2016,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,6652.0,NaN,NaN,NaN,A,46.690,23685.5569,46.690,203020,605.0


In [ ]:
# Filter the DataFrame for GICS Industry codes starting with '20', '25', '35', '45'
df = df[df['gind'].astype(str).str.startswith(('20', '25', '35', '45'))]

# Group by the first two characters of 'gind' and count the rows in each group
industry_counts = df.groupby(df['gind'].astype(str).str[:2]).size()

# Print the result
print(industry_counts)

gind
20    798
25    548
35    685
45    669
dtype: int64


The used industries: Industrials (20), Customer Discretionary (25), Healtcare (35), and Information Technology (45)

In [ ]:
# Filtering to choose rows where "XAD" and "XRD" are not null
df = df.dropna(subset=['xad', 'xrd', 'spcindcd'])

This code filters the DataFrame `df` to keep only the rows where the columns 'xad', 'xrd', and 'spcindcd' do not contain null (NaN) values. It removes any rows where these specific columns have missing data.

In [ ]:
# Display basic information about the dataset
print(df.info())

# Display summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Check for duplicate rows
print(df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 37 to 4681
Data columns (total 42 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gvkey     860 non-null    int64  
 1   datadate  860 non-null    object 
 2   fyear     860 non-null    int64  
 3   indfmt    860 non-null    object 
 4   consol    860 non-null    object 
 5   popsrc    860 non-null    object 
 6   datafmt   860 non-null    object 
 7   tic       860 non-null    object 
 8   conm      860 non-null    object 
 9   curcd     860 non-null    object 
 10  fyr       860 non-null    int64  
 11  at        860 non-null    float64
 12  capx      860 non-null    float64
 13  ceq       860 non-null    float64
 14  ceql      860 non-null    float64
 15  ch        844 non-null    float64
 16  cogs      860 non-null    float64
 17  csho      860 non-null    float64
 18  dt        860 non-null    float64
 19  ebit      860 non-null    float64
 20  emp       860 non-null    floa

This code provides a comprehensive overview of the dataset in `df`. Firstly, it displays basic information such as column data types and non-null counts (`df.info()`). Secondly, it shows summary statistics like mean, standard deviation for numerical columns (`df.describe()`). Lastly, it checks for missing values in each column (`df.isnull().sum()`) and counts the number of duplicate rows in the dataset (`df.duplicated().sum()`).

In [ ]:
import pandas as pd
import numpy as np
# 'PRCC_F', 'CSHO', 'AT', and 'CEQ' columns

# Calculate the numerator part of Tobin's Q: (PRCC_F * CSHO) + AT - CEQ
df['tobins_q_numerator'] = (df['prcc_f'] * df['csho']) + df['at'] - df['ceq']

# Calculate Tobin's Q
df['tobins_q'] = df['tobins_q_numerator'] / df['at']

# Optional: Handle any potential infinite or NaN values
df['tobins_q'].replace([float('inf'), -float('inf')], np.nan, inplace=True)

# Calculate R&D Intensity (R&D expenses / Total assets)
# Replace missing values in 'xrd' and 'sale' with their respective means before calculation
df['xrd'] = pd.to_numeric(df['xrd'], errors='coerce').fillna(df['xrd'].mean())
df['at'] = pd.to_numeric(df['at'], errors='coerce').fillna(df['at'].mean())
df['RnD_Intensity'] = df['xrd'] / df['at']

# Calculate Advertising Intensity (Advertising expenses / Total Assets)
# Replace missing values in 'xad' and 'at' with their respective means before calculation
df['xad'] = pd.to_numeric(df['xad'], errors='coerce').fillna(df['xad'].mean())
df['at'] = pd.to_numeric(df['at'], errors='coerce').fillna(df['at'].mean())
df['Advertising_Intensity'] = df['xad'] / df['at']

# Display the first few rows with the Tobin's Q value
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,costat,prcc_c,mkvalt,prcc_f,gind,spcindcd,tobins_q_numerator,tobins_q,RnD_Intensity,Advertising_Intensity
37,1161,2016-12-31,2016,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,11.34,10602.90,11.34,453010,235.0,13507.90,4.067419,0.303523,0.039446
38,1161,2017-12-31,2017,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,10.28,9940.76,10.28,453010,235.0,12869.76,3.635525,0.327684,0.044068
39,1161,2018-12-31,2018,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,18.46,18552.30,18.46,453010,235.0,21842.30,4.794183,0.314750,0.038630
40,1161,2019-12-31,2019,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,45.86,53656.20,45.86,453010,235.0,56857.20,9.432183,0.256636,0.035999
41,1161,2020-12-31,2020,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,91.71,111060.81,91.71,453010,235.0,114185.81,12.741108,0.221268,0.035037


This code performs calculations on a DataFrame `df`. It calculates Tobin's Q, a financial ratio (the market value of a company divided by the replacement value of its assets), along with R&D Intensity (R&D expenses to total assets ratio) and Advertising Intensity (advertising expenses to total assets ratio), handling missing values by replacing them with the mean of their respective columns. The results, Tobin's Q, R&D Intensity, and Advertising Intensity, are added as new columns to `df`, and the first few rows of the updated DataFrame are displayed.

In [ ]:
# List of columns to be dropped
columns_to_drop = ["xstf", "xstfo", "xstfws", "opili"]

# Dropping the columns from the DataFrame
df.drop(columns=columns_to_drop, inplace=True)

This code snippet defines a list of column names (`columns_to_drop`) that are to be removed from the DataFrame `df`. The specified columns include "xstf", "xstfo", "xstfws", and "opili". It then proceeds to drop these columns from `df`.

In [ ]:
df.fillna(df.mean(), inplace=True)

<ipython-input-9-bf934754a5b1>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [ ]:
# Display basic information about the dataset
print(df.info())

# Display summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())

# Check for duplicate rows
print(df.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 37 to 4681
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gvkey                  860 non-null    int64  
 1   datadate               860 non-null    object 
 2   fyear                  860 non-null    int64  
 3   indfmt                 860 non-null    object 
 4   consol                 860 non-null    object 
 5   popsrc                 860 non-null    object 
 6   datafmt                860 non-null    object 
 7   tic                    860 non-null    object 
 8   conm                   860 non-null    object 
 9   curcd                  860 non-null    object 
 10  fyr                    860 non-null    int64  
 11  at                     860 non-null    float64
 12  capx                   860 non-null    float64
 13  ceq                    860 non-null    float64
 14  ceql                   860 non-null    float64
 15  ch  

This code segment provides a detailed analysis of the DataFrame `df`. It begins by displaying basic information such as column data types and the count of non-null values (`df.info()`). Then, it shows a summary of key statistical measures for each column (`df.describe()`). Finally, it checks and reports the number of missing values per column (`df.isnull().sum()`) and the total count of duplicate rows in the DataFrame (`df.duplicated().sum()`).

In [ ]:
df.head(5)

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,costat,prcc_c,mkvalt,prcc_f,gind,spcindcd,tobins_q_numerator,tobins_q,RnD_Intensity,Advertising_Intensity
37,1161,2016-12-31,2016,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,11.34,10602.90,11.34,453010,235.0,13507.90,4.067419,0.303523,0.039446
38,1161,2017-12-31,2017,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,10.28,9940.76,10.28,453010,235.0,12869.76,3.635525,0.327684,0.044068
39,1161,2018-12-31,2018,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,18.46,18552.30,18.46,453010,235.0,21842.30,4.794183,0.314750,0.038630
40,1161,2019-12-31,2019,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,45.86,53656.20,45.86,453010,235.0,56857.20,9.432183,0.256636,0.035999
41,1161,2020-12-31,2020,INDL,C,D,STD,AMD,ADVANCED MICRO DEVICES,USD,...,A,91.71,111060.81,91.71,453010,235.0,114185.81,12.741108,0.221268,0.035037


In [ ]:
distinct_gvkeys = df['gvkey'].nunique()
print("Distinct gvkey variables:", distinct_gvkeys)

Distinct gvkey variables: 92


In [ ]:
# List of sector codes
sector_codes = ['20', '25', '35', '45']

# Loop through each sector code and count distinct gvkeys
for sector_code in sector_codes:
    # Filter the DataFrame for the current sector code
    filtered_df = df[df['gind'].astype(str).str.startswith(sector_code)]

    # Count the number of distinct gvkey variables in the filtered DataFrame
    distinct_gvkeys = filtered_df['gvkey'].nunique()

    # Print the result for each sector code
    print(f"Number of distinct gvkey variables in sector starting with {sector_code}: {distinct_gvkeys}")


Number of distinct gvkey variables in sector starting with 20: 13
Number of distinct gvkey variables in sector starting with 25: 26
Number of distinct gvkey variables in sector starting with 35: 21
Number of distinct gvkey variables in sector starting with 45: 32


In [ ]:
import pandas as pd

# Assuming df is your DataFrame
# Load or define your DataFrame df here.

# Function to identify duplicate columns
def get_duplicate_columns(df):
    duplicate_columns = []
    for i in range(df.shape[1]):
        col1 = df.iloc[:, i]
        for j in range(i + 1, df.shape[1]):
            col2 = df.iloc[:, j]
            if col1.equals(col2):
                duplicate_columns.append(df.columns.values[j])
    return duplicate_columns

# Get the list of duplicate columns
duplicates = get_duplicate_columns(df)

df.drop(columns=duplicates, inplace=True)

# Print duplicate columns
print("Duplicate Columns:", duplicates)

Duplicate Columns: ['sale']


# Models


## Model 1: Fixed Effects, Tobin's Q, Industry Seperate

- DP: Logged Tobin's Q

- IV's: Logged Advertising Intensity and RnD Intensity

Year Dummy Variables


In [ ]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant

def run_fixed_effects_analysis(df, industry_code):
    # Filter the DataFrame for the specified industry
    df_filtered = df[df['gind'].astype(str).str.startswith(industry_code)]

    # Check if 'gvkey' and 'fyear' are in the columns and set them as index
    if 'gvkey' in df_filtered.columns and 'fyear' in df_filtered.columns:
        df_filtered = df_filtered.set_index(['gvkey', 'fyear'])

    # Create dummy variables for fiscal year and industry categories
    year_dummies = pd.get_dummies(df_filtered.index.get_level_values('fyear'), prefix='Year', drop_first=True)
    df_filtered['industry_category'] = df_filtered['gind'].astype(str).str[:2]
    industry_dummies = pd.get_dummies(df_filtered['industry_category'], prefix='Industry', drop_first=True)

    # Align the indices of the dummy DataFrames with the main DataFrame
    year_dummies.index = df_filtered.index
    industry_dummies.index = df_filtered.index

    # Concatenate dummy variables with the original DataFrame
    df_filtered = pd.concat([df_filtered, year_dummies, industry_dummies], axis=1)

    # Log-transform the dependent variable and key independent variables
    df_filtered['log_tobins_q'] = np.log(df_filtered['tobins_q'].clip(lower=0.01))
    df_filtered['log_RnD_Intensity'] = np.log(df_filtered['RnD_Intensity'].clip(lower=0.01))
    df_filtered['log_Advertising_Intensity'] = np.log(df_filtered['Advertising_Intensity'].clip(lower=0.01))
    df_filtered['log_ni'] = np.log(df_filtered['ni'].clip(lower=0.01))

    # Existing variables and control variables
    existing_vars = ['cogs', 'intan', 'invt', 'ch', 'ceql']
    control_vars = existing_vars + list(industry_dummies.columns) + list(year_dummies.columns)

    # Define the dependent and independent variables for the model
    y1 = df_filtered['log_tobins_q']
    X1 = df_filtered[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]

    # Model 1: Fixed effects with logged variables and robust standard errors
    model_1 = PanelOLS(y1, X1, entity_effects=True, drop_absorbed=True).fit(cov_type='robust')

    # Output the model results with robust standard errors
    print(f"\nFixed Effects Model with Robust Standard Errors for Industry {industry_code}:\n", model_1)

    # Durbin-Watson Test
    residuals = model_1.resids
    dw_statistic = durbin_watson(residuals)
    print(f"Durbin-Watson statistic for Industry {industry_code}:", dw_statistic)

    # Breusch-Pagan Test for Heteroskedasticity
    # Add a constant to X1 for the test
    X1_bp = add_constant(X1.reset_index(drop=True))

    # Perform Breusch-Pagan test
    bp_test = het_breuschpagan(residuals.values, X1_bp)
    labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
    print(f"\nBreusch-Pagan Test for Heteroskedasticity in Industry {industry_code}:\n", dict(zip(labels, bp_test)))

# Run the analysis for each industry separately
for industry in ['20', '25', '35', '45']:
    run_fixed_effects_analysis(df, industry)



Fixed Effects Model with Robust Standard Errors for Industry 20:
                           PanelOLS Estimation Summary                           
Dep. Variable:           log_tobins_q   R-squared:                        0.5470
Estimator:                   PanelOLS   R-squared (Between):             -0.5400
No. Observations:                 133   R-squared (Within):               0.5470
Date:                Sat, Dec 30 2023   R-squared (Overall):             -0.4853
Time:                        05:41:36   Log-likelihood                    65.328
Cov. Estimator:                Robust                                           
                                        F-statistic:                      7.3163
Entities:                          13   P-value                           0.0000
Avg Obs:                       10.231   Distribution:                  F(17,103)
Min Obs:                       4.0000                                           
Max Obs:                       11.000   F-

## Model 2: Fixed Effects, Tobin's Q

- DP: Logged Tobin's Q

- IV's: Logged Advertising Intensity and RnD Intensity

Year Dummy Variables

In [ ]:
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
from statsmodels.stats.stattools import durbin_watson

# Set 'gvkey' and 'fyear' as index if they exist
if 'gvkey' in df.columns and 'fyear' in df.columns:
    df = df.set_index(['gvkey', 'fyear'])

# Create dummy variables for fiscal year
year_dummies = pd.get_dummies(df.index.get_level_values('fyear'), prefix='Year', drop_first=True)

# Create dummy variables for industry categories
df['industry_category'] = df['gind'].astype(str).str[:2]
industry_dummies = pd.get_dummies(df['industry_category'], prefix='Industry', drop_first=True)

# Align the indices of the dummy DataFrames with the main DataFrame
year_dummies.index = df.index
industry_dummies.index = df.index

# Concatenate dummy variables with the original DataFrame
df = pd.concat([df, year_dummies, industry_dummies], axis=1)

# Existing variables
existing_vars = ['cogs', 'intan', 'invt', 'ch', 'ceql']

# Combine existing variables with control variables
control_vars = existing_vars + list(industry_dummies.columns) + list(year_dummies.columns)

# Log-transform the dependent variable and key independent variables if they exist
if 'tobins_q' in df.columns and 'RnD_Intensity' in df.columns and 'Advertising_Intensity' in df.columns and 'ni' in df.columns:
    df['log_tobins_q'] = np.log(df['tobins_q'].clip(lower=0.01))
    df['log_RnD_Intensity'] = np.log(df['RnD_Intensity'].clip(lower=0.01))
    df['log_Advertising_Intensity'] = np.log(df['Advertising_Intensity'].clip(lower=0.01))
    df['log_ni'] = np.log(df['ni'].clip(lower=0.01))

In [ ]:
# Model: Fixed effects with logged variables
X2 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y2 = df['log_tobins_q']
model_2 = PanelOLS(y2, X2, entity_effects=True, drop_absorbed=True).fit()

# Output the model results
print("\nModel 2 (Fixed Effects, IV's → Tobin's Q):\n", model_2)

# Durbin-Watson Test
residuals = model_2.resids
dw_statistic = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X2 for the Breusch-Pagan test
X2_bp = add_constant(X2.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X2_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 2 (Fixed Effects, IV's → Tobin's Q):
                           PanelOLS Estimation Summary                           
Dep. Variable:           log_tobins_q   R-squared:                        0.3104
Estimator:                   PanelOLS   R-squared (Between):             -4.3197
No. Observations:                 860   R-squared (Within):               0.3104
Date:                Sat, Dec 30 2023   R-squared (Overall):             -4.0764
Time:                        05:41:36   Log-likelihood                    13.897
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      19.889
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(17,751)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):   

<ipython-input-17-75620e7832e6>:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Industry_25, Industry_35, Industry_45

  model_2 = PanelOLS(y2, X2, entity_effects=True, drop_absorbed=True).fit()


In [ ]:
# Model: Fixed effects with logged variables using robust standard errors
model_2_robust = PanelOLS(y2, X2, entity_effects=True, drop_absorbed=True).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 2 with Robust Standard Errors (Fixed Effects, IV's → Tobin's Q):\n", model_2_robust)


Updated Model 2 with Robust Standard Errors (Fixed Effects, IV's → Tobin's Q):
                           PanelOLS Estimation Summary                           
Dep. Variable:           log_tobins_q   R-squared:                        0.3104
Estimator:                   PanelOLS   R-squared (Between):             -4.3197
No. Observations:                 860   R-squared (Within):               0.3104
Date:                Sat, Dec 30 2023   R-squared (Overall):             -4.0764
Time:                        05:41:37   Log-likelihood                    13.897
Cov. Estimator:                Robust                                           
                                        F-statistic:                      19.889
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(17,751)
Min Obs:                       1.0000                                           
Max Obs:                    

<ipython-input-18-2e79082a0f4a>:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Industry_25, Industry_35, Industry_45

  model_2_robust = PanelOLS(y2, X2, entity_effects=True, drop_absorbed=True).fit(cov_type='robust')


In [ ]:
import scipy.stats as stats

# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: 0.18299287331374542
Kurtosis of residuals: 4.3592253365207485


In this code, dummy variables are created for fiscal years and industry categories to include in a random effects regression model. The `year_dummies` are generated from the 'fyear' field in the DataFrame's index, creating a separate column for each year with a prefix 'Year', and the first category is dropped to avoid multicollinearity. For industry categories, the first two digits of the GICS code in the 'gind' column are extracted to create `industry_dummies`, with each unique industry category represented by a new column prefixed with 'Industry'. These dummy variables are aligned with the main DataFrame's index and then concatenated to `df`, expanding its feature set. Finally, these dummies, along with a set of existing financial variables (`existing_vars`), are used as control and independent variables in a random effects model (`model_2`) to predict Tobin's Q, a measure of a company's market value relative to its assets.

## Model 3: Fixed Effects, Net Income,

- DP: Logged Net Income

- IV's: Logged Advertising Intensity and RnD Intensity

Year Dummy Variables

In [ ]:
# Model: Fixed effects with logged variables
X3 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y3 = df['log_ni']
model_3 = PanelOLS(y3, X3, entity_effects=True, drop_absorbed=True).fit()

# Output the model results
print("\nModel 3 (Fixed Effects, IV's → Net Income):\n", model_3)

# Durbin-Watson Test
residuals = model_3.resids
dw_statistic = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X3 for the Breusch-Pagan test
X3_bp = add_constant(X3.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X3_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 3 (Fixed Effects, IV's → Net Income):
                           PanelOLS Estimation Summary                           
Dep. Variable:                 log_ni   R-squared:                        0.0595
Estimator:                   PanelOLS   R-squared (Between):              0.5419
No. Observations:                 860   R-squared (Within):               0.0595
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.4727
Time:                        05:41:37   Log-likelihood                   -1955.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.7955
Entities:                          92   P-value                           0.0001
Avg Obs:                       9.3478   Distribution:                  F(17,751)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):  

<ipython-input-20-2b309bc2e319>:4: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Industry_25, Industry_35, Industry_45

  model_3 = PanelOLS(y3, X3, entity_effects=True, drop_absorbed=True).fit()


In [ ]:
# Model: Fixed effects with logged variables using robust standard errors
model_3_robust = PanelOLS(y3, X3, entity_effects=True, drop_absorbed=True).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 3 with Robust Standard Errors (Fixed Effects, IV's → Net Income):\n", model_3_robust)

<ipython-input-21-43ff06f02ab4>:2: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Industry_25, Industry_35, Industry_45

  model_3_robust = PanelOLS(y3, X3, entity_effects=True, drop_absorbed=True).fit(cov_type='robust')



Updated Model 3 with Robust Standard Errors (Fixed Effects, IV's → Net Income):
                           PanelOLS Estimation Summary                           
Dep. Variable:                 log_ni   R-squared:                        0.0595
Estimator:                   PanelOLS   R-squared (Between):              0.5419
No. Observations:                 860   R-squared (Within):               0.0595
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.4727
Time:                        05:41:37   Log-likelihood                   -1955.7
Cov. Estimator:                Robust                                           
                                        F-statistic:                      2.7955
Entities:                          92   P-value                           0.0001
Avg Obs:                       9.3478   Distribution:                  F(17,751)
Min Obs:                       1.0000                                           
Max Obs:                   

In [ ]:
# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: -2.149142366359158
Kurtosis of residuals: 10.676923584097256


## VIF Analysis

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Calculate VIF for each explanatory variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Display the VIF for each variable
print(vif_data)

                     Variable         VIF
0                       const  100.349044
1           log_RnD_Intensity    2.112001
2   log_Advertising_Intensity    1.277270
3                        cogs    4.233688
4                       intan    2.445030
5                        invt    4.281438
6                          ch    2.425235
7                        ceql    3.037067
8                 Industry_25    2.393512
9                 Industry_35    2.522521
10                Industry_45    3.087682
11                  Year_2013    2.108209
12                  Year_2014    2.095182
13                  Year_2015    2.070643
14                  Year_2016    2.034489
15                  Year_2017    2.047398
16                  Year_2018    2.063166
17                  Year_2019    2.047935
18                  Year_2020    2.075343
19                  Year_2021    2.062493
20                  Year_2022    1.903412


This code snippet calculates the Variance Inflation Factor (VIF) for each explanatory variable in a DataFrame `df` to assess multicollinearity. It first selects a set of independent variables, 'RnD_Intensity', 'Advertising_Intensity', and additional control variables, and then adds a constant term to represent the intercept in the regression model. The VIF for each variable is computed using the `variance_inflation_factor` function from `statsmodels` and stored in a new DataFrame `vif_data`, which is then displayed to show the multicollinearity level of each explanatory variable in the model.

## Model 4: Random Effects, Tobin's Q

- DP: Logged Tobin's Q

- IV's: Logged Advertising Intensity and RnD Intensity

Year and Industry Dummy Variables

In [ ]:
from linearmodels.panel import RandomEffects

# Model: Random effects with logged variables
X4 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y4 = df['log_tobins_q']
model_4 = RandomEffects(y4, X4).fit()

# Output the model results
print("\nModel 4 (Random Effects, IV's → tobins_q):\n", model_4)

# Durbin-Watson Test
residuals = model_4.resids
dw_statistic = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X4 for the Breusch-Pagan test
X4_bp = add_constant(X4.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X4_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 4 (Random Effects, IV's → tobins_q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4519
Estimator:              RandomEffects   R-squared (Between):              0.8199
No. Observations:                 860   R-squared (Within):               0.2521
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.7759
Time:                        05:41:38   Log-likelihood                   -74.211
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      34.634
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(20,840)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):   

In [ ]:
# Model: Random effects with logged variables using robust standard errors
model_4_robust = RandomEffects(y4, X4).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 4 with Robust Standard Errors (Random Effects, IV's → Tobin's Q):\n", model_4_robust)


Updated Model 4 with Robust Standard Errors (Random Effects, IV's → Tobin's Q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4519
Estimator:              RandomEffects   R-squared (Between):              0.8199
No. Observations:                 860   R-squared (Within):               0.2521
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.7759
Time:                        05:41:38   Log-likelihood                   -74.211
Cov. Estimator:                Robust                                           
                                        F-statistic:                      34.634
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(20,840)
Min Obs:                       1.0000                                           
Max Obs:                   

In [ ]:
# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: 0.323938522521126
Kurtosis of residuals: 3.6244774913388493


## Model 5: Random Effects, Net Income

- DP: Logged Net Income

- IV's: Logged Advertising Intensity and RnD Intensity

Year and Industry Dummy Variables

In [ ]:
from linearmodels.panel import RandomEffects

# Model: Random effects with logged variables
X5 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y5 = df['log_ni']
model_5 = RandomEffects(y5, X5).fit()

# Output the model results
print("\nModel 5 (Random Effects, IV's → Net Income):\n", model_5)

# Durbin-Watson Test
residuals = model_5.resids
dw_statistic = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X5 for the Breusch-Pagan test
X5_bp = add_constant(X5.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X5_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 5 (Random Effects, IV's → Net Income):
                         RandomEffects Estimation Summary                        
Dep. Variable:                 log_ni   R-squared:                        0.5930
Estimator:              RandomEffects   R-squared (Between):              0.9268
No. Observations:                 860   R-squared (Within):               0.0199
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8226
Time:                        05:41:38   Log-likelihood                   -2024.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      61.205
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(20,840)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust): 

In [ ]:
# Model: Random effects with logged variables using robust standard errors
model_5_robust = RandomEffects(y5, X5).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 5 with Robust Standard Errors (Random Effects, IV's → Net Income):\n", model_5_robust)


Updated Model 5 with Robust Standard Errors (Random Effects, IV's → Net Income):
                         RandomEffects Estimation Summary                        
Dep. Variable:                 log_ni   R-squared:                        0.5930
Estimator:              RandomEffects   R-squared (Between):              0.9268
No. Observations:                 860   R-squared (Within):               0.0199
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8226
Time:                        05:41:39   Log-likelihood                   -2024.4
Cov. Estimator:                Robust                                           
                                        F-statistic:                      61.205
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(20,840)
Min Obs:                       1.0000                                           
Max Obs:                  

In [ ]:
# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: -2.7698505725590885
Kurtosis of residuals: 11.576669125239054


# Interaction Models

## Data Preparation


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Create interaction terms between Advertising_Intensity and industry categories directly in df
for industry_code in df['industry_category'].unique():
    industry_dummy = 'Industry_' + industry_code
    df[industry_dummy] = (df['industry_category'] == industry_code).astype(int)
    interaction_term = f'adv_intensity_x_{industry_code}'
    df[interaction_term] = df['Advertising_Intensity'] * df[industry_dummy]
    control_vars.append(interaction_term)

# Drop the interaction term with industry code 20
df.drop('adv_intensity_x_20', axis=1, inplace=True)
if 'adv_intensity_x_20' in control_vars:
    control_vars.remove('adv_intensity_x_20')

# Define your independent variables for the model
XX = df[['RnD_Intensity', 'Advertising_Intensity'] + control_vars]

In [ ]:
# Model 6: Combined Effect of IVs and their interactions with Industry on Tobin's Q
X6 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y6 = df['log_tobins_q']
model_6 = RandomEffects(y6, X6).fit()
print("\nModel 6 (IVs and Advertising-Industry Interactions → tobins_q):\n", model_6)

# Extract residuals from the fitted model
residuals = model_6.resids

# Perform Durbin-Watson test
dw_statistic = durbin_watson(residuals)

# Output the Durbin-Watson statistic
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X6 for the Breusch-Pagan test
X6_bp = add_constant(X6.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X6_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 6 (IVs and Advertising-Industry Interactions → tobins_q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4875
Estimator:              RandomEffects   R-squared (Between):              0.8614
No. Observations:                 860   R-squared (Within):               0.2866
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8205
Time:                        05:41:39   Log-likelihood                   -43.575
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      34.611
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(23,837)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-s

## Model 6: Random Effects, Tobin's Q, Advertising Intensity Interaction

- DP: Logged Tobin's Q

- IV's: Logged Advertising Intensity and RnD Intensity

Year, Industry, and Interaction Dummy Variables

In [ ]:
# Model 6: Combined Effect of IVs and their interactions with Industry on Tobin's Q
# Using robust standard errors
model_6_robust = RandomEffects(y6, X6).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 6 with Robust Standard Errors (IVs and Advertising-Industry Interactions → Tobin's Q):\n", model_6_robust)


Updated Model 6 with Robust Standard Errors (IVs and Advertising-Industry Interactions → Tobin's Q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4875
Estimator:              RandomEffects   R-squared (Between):              0.8614
No. Observations:                 860   R-squared (Within):               0.2866
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8205
Time:                        05:41:39   Log-likelihood                   -43.575
Cov. Estimator:                Robust                                           
                                        F-statistic:                      34.611
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(23,837)
Min Obs:                       1.0000                                           
Max Ob

In [ ]:
# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: 0.3946065789720247
Kurtosis of residuals: 3.8446615584503196


## Model 7: Random Effects, Tobin's Q, RND Intensity Interaction

- DP: Logged Tobin's Q

- IV's: Logged Advertising Intensity and RnD Intensity

Year, Industry, and Interaction Dummy Variables

In [ ]:
control_vars

['cogs',
 'intan',
 'invt',
 'ch',
 'ceql',
 'Industry_25',
 'Industry_35',
 'Industry_45',
 'Year_2013',
 'Year_2014',
 'Year_2015',
 'Year_2016',
 'Year_2017',
 'Year_2018',
 'Year_2019',
 'Year_2020',
 'Year_2021',
 'Year_2022',
 'adv_intensity_x_45',
 'adv_intensity_x_35',
 'adv_intensity_x_25']

In [ ]:
# Remove advertising intensity interaction terms
control_vars = [var for var in control_vars if not var.startswith('adv_intensity_x_')]

In [ ]:
# Create interaction terms between RnD_Intensity and industry categories directly in df
for industry_code in df['industry_category'].unique():
    industry_dummy = 'Industry_' + industry_code
    df[industry_dummy] = (df['industry_category'] == industry_code).astype(int)
    interaction_term = f'RnD_intensity_x_{industry_code}'
    df[interaction_term] = df['RnD_Intensity'] * df[industry_dummy]
    if industry_code != '20':  # Exclude the interaction with industry code 20
        control_vars.append(interaction_term)

# Drop the interaction term with industry code 20 if it was created
interaction_term_20 = 'RnD_intensity_x_20'
if interaction_term_20 in df.columns:
    df.drop(interaction_term_20, axis=1, inplace=True)
if interaction_term_20 in control_vars:
    control_vars.remove(interaction_term_20)


In [ ]:
# Model 6: Combined Effect of RnD_Intensity, its interactions with Industry, and other IVs on Tobin's Q
X7 = df[['log_RnD_Intensity', 'log_Advertising_Intensity'] + control_vars]
y7 = df['log_tobins_q']
model_7 = RandomEffects(y7, X7).fit()
print("\nModel 7 (RnD Intensity and its Industry Interactions → tobins_q):\n", model_7)

# Extract residuals from the fitted model
residuals = model_7.resids

# Perform Durbin-Watson test
dw_statistic = durbin_watson(residuals)

# Output the Durbin-Watson statistic
print("Durbin-Watson statistic:", dw_statistic)

# Add a constant to X7 for the Breusch-Pagan test
X7_bp = add_constant(X7.reset_index(drop=True))

# Perform Breusch-Pagan test
bp_test = het_breuschpagan(residuals.values, X7_bp)
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
print("\nBreusch-Pagan Test for Heteroskedasticity:\n", dict(zip(labels, bp_test)))


Model 7 (RnD Intensity and its Industry Interactions → tobins_q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4789
Estimator:              RandomEffects   R-squared (Between):              0.8519
No. Observations:                 860   R-squared (Within):               0.2778
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8122
Time:                        05:41:40   Log-likelihood                   -51.209
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      33.447
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(23,837)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F

In [ ]:
# Model 7: Combined Effect of RnD_Intensity, its interactions with Industry, and other IVs on Tobin's Q
# Updated to use robust standard errors
model_7_robust = RandomEffects(y7, X7).fit(cov_type='robust')

# Output the updated model results with robust standard errors
print("\nUpdated Model 7 with Robust Standard Errors (RnD Intensity and its Industry Interactions → Tobin's Q):\n", model_7_robust)


Updated Model 7 with Robust Standard Errors (RnD Intensity and its Industry Interactions → Tobin's Q):
                         RandomEffects Estimation Summary                        
Dep. Variable:           log_tobins_q   R-squared:                        0.4789
Estimator:              RandomEffects   R-squared (Between):              0.8519
No. Observations:                 860   R-squared (Within):               0.2778
Date:                Sat, Dec 30 2023   R-squared (Overall):              0.8122
Time:                        05:41:40   Log-likelihood                   -51.209
Cov. Estimator:                Robust                                           
                                        F-statistic:                      33.447
Entities:                          92   P-value                           0.0000
Avg Obs:                       9.3478   Distribution:                  F(23,837)
Min Obs:                       1.0000                                           
Max 

In [ ]:
# Calculate skewness
skewness = stats.skew(residuals)

# Calculate kurtosis
kurtosis = stats.kurtosis(residuals, fisher=False)

print("Skewness of residuals:", skewness)
print("Kurtosis of residuals:", kurtosis)

Skewness of residuals: 0.34900412851539947
Kurtosis of residuals: 3.90895715613366
